In [61]:
#Load necessary packages
import tweepy
import folium
import geopy
import os
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
import numpy as np

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#set authentication variables
access_token_eve = os.environ.get('access_token')
access_secret_eve = os.environ.get('access_secret')
consumer_key_eve = os.environ.get('api_key')
consumer_secret_eve = os.environ.get('api_secret')

In [ ]:
#Set inital data frame to store tweets for given time frame
#Pull tweet date/time


In [6]:
#load the Twitter API, create def for use
def get_twitter_api():
    # personal details
    consumer_key = consumer_key_eve
    consumer_secret = consumer_secret_eve
    access_token = access_token_eve
    access_token_secret = access_secret_eve
    
    # authentication of consumer key and secret
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    
    # authentication of access token and secret
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    
    return api

#set up search for coordinates and/or location of the tweets
def get_twitter_location(search):
    api = get_twitter_api()
    count = 0
    for tweet in tweepy.Cursor(api.search_tweets, q=search).items(500):
        if hasattr(tweet, 'coordinates') and tweet.coordinates is not None:
            count += 1
            print("Coordinates", tweet.coordinates)
        if hasattr(tweet, 'location') and tweet.location is not None:
            count += 1
            print("Coordinates", tweet.location)
    print(count)
get_twitter_location("#Everton")

0


In [41]:
#set search for location of the Twitter user
#incorporate Twitter's own Lon/Lat data into the equation:
#https://developer.twitter.com/en/docs/tutorials/filtering-tweets-by-location

def get_tweets(search):
    api = get_twitter_api()
    location_data = []
    for tweet in tweepy.Cursor(api.search_tweets, q=search).items(15):
        if hasattr(tweet, 'user') and hasattr(tweet.user, 'screen_name') and hasattr(tweet.user, 'location') and hasattr(tweet, 'created_at') and hasattr(tweet, 'id'):
            if tweet.user.location:
                location_data.append((tweet.id, tweet.user.screen_name, tweet.user.location, str(tweet.created_at)))
    return location_data

In [42]:
#pull the tweets and store in data frame 
#data should include -Twitter screen name; location, created date, Tweet ID
search_terms="#Everton"
tweet_batch = get_tweets(search_terms)
print(tweet_batch)

[(1547864797005225991, 'ScribblerBlue', 'Liverpool, England', '2022-07-15 08:45:00+00:00'), (1547864545330401280, 'ScribblerBlue', 'Liverpool, England', '2022-07-15 08:44:00+00:00'), (1547864076394438657, 'chriswatson187', 'Exiled Blue living in Glasgow ', '2022-07-15 08:42:08+00:00'), (1547864061747949569, 'UHTPodcast', 'Liverpool, England', '2022-07-15 08:42:04+00:00'), (1547863790351532032, 'ScribblerBlue', 'Liverpool, England', '2022-07-15 08:41:00+00:00'), (1547862832988823555, 'RiddledBlues2', 'North Sentinal Island', '2022-07-15 08:37:11+00:00'), (1547862280708055041, 'ScribblerBlue', 'Liverpool, England', '2022-07-15 08:35:00+00:00'), (1547861870874222593, 'GregOK', 'North West, England', '2022-07-15 08:33:22+00:00'), (1547860518941106176, 'ScribblerBlue', 'Liverpool, England', '2022-07-15 08:28:00+00:00')]


In [46]:
#convert array to NumPy array
tweets=np.array(tweet_batch)
print(tweets)

[['1547864797005225991' 'ScribblerBlue' 'Liverpool, England'
  '2022-07-15 08:45:00+00:00']
 ['1547864545330401280' 'ScribblerBlue' 'Liverpool, England'
  '2022-07-15 08:44:00+00:00']
 ['1547864076394438657' 'chriswatson187' 'Exiled Blue living in Glasgow '
  '2022-07-15 08:42:08+00:00']
 ['1547864061747949569' 'UHTPodcast' 'Liverpool, England'
  '2022-07-15 08:42:04+00:00']
 ['1547863790351532032' 'ScribblerBlue' 'Liverpool, England'
  '2022-07-15 08:41:00+00:00']
 ['1547862832988823555' 'RiddledBlues2' 'North Sentinal Island'
  '2022-07-15 08:37:11+00:00']
 ['1547862280708055041' 'ScribblerBlue' 'Liverpool, England'
  '2022-07-15 08:35:00+00:00']
 ['1547861870874222593' 'GregOK' 'North West, England'
  '2022-07-15 08:33:22+00:00']
 ['1547860518941106176' 'ScribblerBlue' 'Liverpool, England'
  '2022-07-15 08:28:00+00:00']]


In [50]:
#convert to dataframe
df = pd.DataFrame(tweets, columns=["ID", "Name", "Location", "Date-Time"])
print(df.head())

                    ID            Name                        Location  \
0  1547864797005225991   ScribblerBlue              Liverpool, England   
1  1547864545330401280   ScribblerBlue              Liverpool, England   
2  1547864076394438657  chriswatson187  Exiled Blue living in Glasgow    
3  1547864061747949569      UHTPodcast              Liverpool, England   
4  1547863790351532032   ScribblerBlue              Liverpool, England   

                   Date-Time  
0  2022-07-15 08:45:00+00:00  
1  2022-07-15 08:44:00+00:00  
2  2022-07-15 08:42:08+00:00  
3  2022-07-15 08:42:04+00:00  
4  2022-07-15 08:41:00+00:00  


In [64]:
def addLatLon(data):
    geo_locator = Nominatim(user_agent="EvertonTweeters")
    geocode = RateLimiter(geo_locator.geocode, min_delay_seconds=1)
    tweet_loc = df.copy()
    loc = tweet_loc["Location"].apply(lambda location: geocode(location))
    tweet_loc['latitude'] = [x.latitude for x in loc]
    tweet_loc['longitude'] = [x.longitude for x in loc]
    return tweet_loc

In [ ]:
addLatLon(df)
print(tweet_loc)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Liverpool, England',), **{}).
Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 386, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 1040, in _validate_conn
    conn.connect()
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connection.py", line 414, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\ssl.py", line 512, in wra

RateLimiter swallowed an error after 2 retries. Called with (*('Liverpool, England',), **{}).
Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 386, in _make_request
    self._validate_conn(

RateLimiter caught an error, retrying (1/2 tries). Called with (*('Liverpool, England',), **{}).
Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 386, in _make_request
    self._validate_co

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Exiled Blue living in Glasgow ',), **{}).
Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 386, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 1040, in _validate_conn
    conn.connect()
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connection.py", line 414, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\ssl.py", line

RateLimiter swallowed an error after 2 retries. Called with (*('Exiled Blue living in Glasgow ',), **{}).
Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 386, in _make_request
    self._va

...

RateLimiter caught an error, retrying (1/2 tries). Called with (*('North Sentinal Island',), **{}).
Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\socket.py", line 705, in readinto
    return self.

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Liverpool, England',), **{}).
Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 386, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 1040, in _validate_conn
    conn.connect()
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connection.py", line 414, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\ssl.py", line 512, in wra

RateLimiter swallowed an error after 2 retries. Called with (*('Liverpool, England',), **{}).
Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 386, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 1040, in _validate_conn
    conn.connect()
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connection.py", line 414, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\ssl.py", line 512, in wrap_s

RateLimiter caught an error, retrying (1/2 tries). Called with (*('North West, England',), **{}).
Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 386, in _make_request
    self._validate_c

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Liverpool, England',), **{}).
Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\socket.py", line 705, in readinto
    return self._so

In [49]:
def put_markers(map, data):
    geo_locator = Nominatim(user_agent="EvertonTweeters")
    for (ID, name, location, time) in data:
        if location:
            try:
                location = geo_locator.geocode(location)
            except GeocoderTimedOut:
                continue
            if location:
                folium.Marker([location.latitude, location.longitude], popup=name).add_to(map)

if __name__ == "__main__":
    map = folium.Map(location=[0, 0], zoom_start=2)
    location_data = tweets
    put_markers(map, location_data)
    map.save("index.html")

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Liverpool%2C+England&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))